### 导入模块

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### load data

In [ ]:
test = pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv")
item_categories = pd.read_csv("../input/competitive-data-science-predict-future-sales/item_categories.csv")
items = pd.read_csv("../input/competitive-data-science-predict-future-sales/items.csv")
sales_train =  pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv")
sample_submission =  pd.read_csv("../input/competitive-data-science-predict-future-sales/sample_submission.csv")
shops = pd.read_csv("../input/competitive-data-science-predict-future-sales/shops.csv")

### data explor

In [ ]:
item_categories.head(5)

In [ ]:
items.head(5)

In [ ]:
shops.head(5)

In [ ]:
sales_train.head(5)

In [ ]:
sample_submission.head(5)

In [ ]:
test.head(5)

#### check the outliers

In [ ]:
# 得到前面的问题的答案
# 1、先看下信息完整不
sales_train.head(3)

In [ ]:
sales_train.describe()

In [ ]:
sales_train.info(show_counts=True)

没有缺失值

In [ ]:
MONTHS = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
LINEWIDTH=2
ALPHA=.6

# 弄一个copy
sales_copy = sales_train[['date', 'date_block_num', 'item_cnt_day']].copy()
# 处理日期信息
sales_copy['date'] = pd.to_datetime(sales_copy['date'], format = '%d.%m.%Y')# 注意Y大学，其他小写
sales_copy['year'] = sales_copy['date'].dt.year
sales_copy['month'] = sales_copy['date'].dt.month
sales_copy.drop(['date'], axis = 1, inplace=True)

# 按照年与月聚合数据
sales_copy = sales_copy.groupby(['date_block_num'], as_index=False).agg({'year':'first', 'month':'first', 'item_cnt_day':'sum'}).rename(columns={'item_cnt_day':'item_cnt_month'}, inplace=False)



In [ ]:
# 绘图
plt.figure(figsize=(16,6))
plt.plot(MONTHS, sales_copy[sales_copy.year==2013].item_cnt_month, '-o', color='steelblue', linewidth=LINEWIDTH, alpha=ALPHA,label='2013')
# Plot the sales of the year 2014
plt.plot(MONTHS, sales_copy[sales_copy.year==2014].item_cnt_month, '-o', color='seagreen', linewidth=LINEWIDTH, alpha=ALPHA,label='2014')

# Plot the sales of the year 2015 until October
plt.plot(MONTHS[:10], sales_copy[sales_copy.year==2015].item_cnt_month, '-o', color='maroon', linewidth=LINEWIDTH, alpha=ALPHA,label='2015')

ax = plt.gca()
ax.set_title('Sales per month')
ax.set_ylabel('# of items')
ax.grid(axis='y', color='gray', alpha=.2)
    
# 移除边框
for spine in plt.gca().spines.values():
    spine.set_visible(False)
plt.legend(loc=2, title='Legend')
plt.show()

#### 可以看出存在明显的趋势，需要预测的是2015年11月的数据
#### 同时，销售量每年都在下降

In [ ]:
sales_copy

In [ ]:
# 看下销售额的情况
sales_copy = sales_train[['date', 'date_block_num', 'item_cnt_day','item_price']].copy()
# 处理日期信息
sales_copy['date'] = pd.to_datetime(sales_copy['date'], format = '%d.%m.%Y')# 注意Y大学，其他小写
sales_copy['year'] = sales_copy['date'].dt.year
sales_copy['month'] = sales_copy['date'].dt.month
sales_copy.drop(['date'], axis = 1, inplace=True)
sales_copy['item_price_day'] = sales_copy['item_cnt_day'] * sales_copy['item_price']
# 按照年与月聚合数据
sales_copy = sales_copy.groupby(['date_block_num'], as_index=False).agg({'year':'first', 'month':'first', 'item_cnt_day':'sum', 'item_price_day': 'sum'}).rename(columns={'item_cnt_day':'item_cnt_month', 'item_price_day':'item_price_month'}, inplace=False)



In [ ]:
# 绘图
plt.figure(figsize=(16,6))
plt.plot(MONTHS, sales_copy[sales_copy.year==2013].item_price_month, '-o', color='steelblue', linewidth=LINEWIDTH, alpha=ALPHA,label='2013')
# Plot the sales of the year 2014
plt.plot(MONTHS, sales_copy[sales_copy.year==2014].item_price_month, '-o', color='seagreen', linewidth=LINEWIDTH, alpha=ALPHA,label='2014')

# Plot the sales of the year 2015 until October
plt.plot(MONTHS[:10], sales_copy[sales_copy.year==2015].item_price_month, '-o', color='maroon', linewidth=LINEWIDTH, alpha=ALPHA,label='2015')

ax = plt.gca()
ax.set_title('Sales amount per month')
ax.set_ylabel('# of items')
ax.grid(axis='y', color='gray', alpha=.2)
    
# 移除边框
for spine in plt.gca().spines.values():
    spine.set_visible(False)
plt.legend(loc=2, title='Legend')
plt.show()

#### 销售金额2015年也有下降的趋势

In [ ]:
print(item_categories.columns)
print(items.columns)

In [ ]:
# 定义参数
N = 20 # top销量的商品

# 绘制每年销量最高的商品
product_sale = sales_train[['date','date_block_num','item_id','item_cnt_day']].copy()
product_sale['date'] = pd.to_datetime(product_sale['date'],format='%d.%m.%Y')
product_sale['year'] = product_sale['date'].dt.year
product_sale['month'] = product_sale['date'].dt.month
product_sale.drop(['date'], axis = 1, inplace=True)
# 移除退货产品
product_sale = product_sale[product_sale['item_cnt_day']>0]
# 合并类别到商品表种
cat = pd.merge(items, item_categories, how = 'left', on = 'item_category_id')[['item_id','item_category_name']]
product_sale = product_sale.merge(cat,how = 'left', on = 'item_id')

product_sale = product_sale.groupby(['year','item_category_name'],as_index=False).agg({'item_cnt_day':'sum'}).rename(columns={'item_cnt_day':'item_cnt_year'}, inplace=False)
product_sale.item_cnt_year = product_sale.item_cnt_year.astype('int')

# top
top = product_sale.groupby('year',group_keys=False).apply(lambda x:x.nlargest(N, 'item_cnt_year'))

In [ ]:
years = [2013, 2014, 2015]
fig, axes = plt.subplots(1, 3, figsize=(16,6))

def get_ratio(year, topn, N):
    # Get total sold items for each year
    total = product_sale[product_sale['year']==year].item_cnt_year.sum()
    ratio = topn/total*100
    return "{0}: the total of the top {1} best selling items is {2} over a total of {3} for that year, which represents {4:.2f}%".format(year, N, topn, total, ratio)

#Prepare colors for the top N
colors = [[] for i in range(3)]
for alpha in np.arange(N, 0, -1)/N:
    colors[0].append((.275, .51, .706, alpha))
    colors[1].append((.18, .55, .34, alpha))
    colors[2].append((.5, 0, 0, alpha))
    
for ax, year, cs in zip(axes, years, colors):
    # Get top items for each year
    year_filter = top[top.year==year]
    plot_sizes = year_filter.item_cnt_year # 每年的商品数量
    plot_labels = year_filter.item_category_name.str[:15]#+'('+plot_sizes.astype(str)+')' #截取前15个字符的标签
    
    # Get the ratio
    print(get_ratio(year, plot_sizes.sum(), N)) #输出信息
    
    # Plot the pie
    ax.pie(plot_sizes, labels=plot_labels, radius=1.5, colors=cs,labeldistance=.5, rotatelabels=True, startangle=90, wedgeprops={"edgecolor":"1",'linewidth': .5})
    # Set titles below pies
    ax.set_title(year, y=-0.2)

# Space pies
fig.tight_layout()
fig.suptitle('Top selling categories for each year', fontsize=16)
plt.show()

### Detecting outliers in item_price and item_cnt_day

In [ ]:
fig, axes = plt.subplots(2, 1)
plt.subplots_adjust(hspace=0.5)

flierprops = dict(marker='o', markerfacecolor='cornflowerblue', markersize=6, markeredgecolor='navy')

_ = axes[0].boxplot(x=sales_train.item_cnt_day, flierprops=flierprops, vert=False)
_ = axes[1].boxplot(x=sales_train.item_price, flierprops=flierprops, vert=False)

_ = axes[0].set_title('item_cnt_day')
_ = axes[1].set_title('item_price')

#### 存在异常值，所以先看看异常值

In [ ]:
sales_train.columns

In [ ]:
# 销量大于2000的
sales_train[sales_train['item_cnt_day']>2000]

In [ ]:
# 查看一下，该商品的销量情况
sales_train[sales_train['item_id']==11373][sales_train['item_cnt_day'] < 2000]['item_cnt_day'].mean()

粗布判断是异常值！


In [ ]:
# 看下价格的异常值
sales_train[sales_train['item_price']>100000]

In [ ]:
items.columns

In [ ]:
items[items['item_id']==6066]

In [ ]:
item_categories[item_categories['item_category_id']==75]

In [ ]:
sales_train[sales_train['item_id']>6066]

#### 也是异常值 可以删除

In [ ]:
train_data = sales_train.copy()

In [ ]:
train_data.drop([2909818,1163158],axis = 0, inplace=True)

In [ ]:
train_data

In [ ]:

plt.boxplot(x=train_data.item_cnt_day, flierprops=flierprops, vert=False)


In [ ]:
plt.boxplot(x=train_data.item_price, flierprops=flierprops, vert=False)

#### 整理下数据，按月份与商品名称 商店名称聚合数据


In [ ]:
train_data.columns

In [ ]:
train_data['date'] = pd.to_datetime(train_data['date'],format='%d.%m.%Y')
train_data['year'] = train_data.date.dt.year
train_data['month'] = train_data.date.dt.month
train_data['total_price'] = train_data['item_price'] * train_data['item_cnt_day']
train_data = train_data.groupby(['date_block_num', 'shop_id', 'item_id'],as_index=False).agg({'year':'first', 'month':'first', 'item_price':np.mean,'item_cnt_day':np.sum,'total_price':np.sum}).rename({'item_cnt_day':'item_cnt_month'})


In [ ]:
train_data

In [ ]:
# 查看预测的商品占据比重
print(test.columns)
print(items.columns)

In [ ]:
print("Test products have {}% in total products" .format(round(float(len(np.unique(test['item_id']))/len(items.item_id.tolist()))*100,3)))
print("Train products have {}% in total products on 2013" .format(round(float(len(np.unique(train_data[train_data['year']==2013]['item_id']))/len(items.item_id.tolist()))*100,3)))
print("Train products have {}% in total products on 2014" .format(round(float(len(np.unique(train_data[train_data['year']==2014]['item_id']))/len(items.item_id.tolist()))*100,3)))
print("Train products have {}% in total products on 2015" .format(round(float(len(np.unique(train_data[train_data['year']==2015]['item_id']))/len(items.item_id.tolist()))*100,3)))

In [ ]:
# 先测试下普通的线性回归
# 先筛选商品
test.info(show_counts=True)

In [ ]:
# 得到对应shop_id与item_id的数据
test["combin"] = test.apply(lambda x: str(int(x["shop_id"]))+str(int(x["item_id"])), axis = 1)
train_data["combin"] = train_data.apply(lambda x: str(int(x["shop_id"]))+str(int(x["item_id"])), axis = 1)
train_data_test = train_data[train_data["combin"].isin(set(test["combin"]))].copy()

In [ ]:
# 按月去整理每月每个店铺的这些商品的销量
# 用merge吧
test_temp = test[["shop_id","item_id","combin"]].copy().drop_duplicates(['combin'])
test_temp["item_cnt_day"] = 0
date_list = list(set(train_data_test["date_block_num"]))
columns_list = ["shop_id","item_id","item_cnt_day","combin"]
data_train_MoveAverage = pd.DataFrame(columns=("date_block_num","shop_id","item_id","item_cnt_day","combin"))
for mo in date_list:
    data_temp = train_data_test[train_data_test["date_block_num"]==mo][columns_list].drop_duplicates(['combin'])# 少列
    need_add = test_temp[~test_temp["combin"].isin(data_temp["combin"])]
    data_new = pd.concat([data_temp,need_add],axis=0)
    data_new["date_block_num"] = mo
    data_train_MoveAverage = pd.concat([data_train_MoveAverage,data_new],axis=0)
data_train_MoveAverage

In [ ]:
# α指数平滑预测
# alpha = 0.3
# def predict_test(alpha):
#     data_Next = np.array(data_train_MoveAverage[data_train_MoveAverage["date_block_num"]==0].sort_values(by=['combin'])["item_cnt_day"].astype('float32'))
#     for t in range(1,33,1):
#         data_Now = np.array(data_train_MoveAverage[data_train_MoveAverage["date_block_num"]==t].sort_values(by=['combin'])["item_cnt_day"].astype('float32'))
#         data_Next = (1-alpha) * data_Next + alpha * data_Now
#     # data_Next
# #     pre_test = test.merge(data_Next,how = 'left',on = ['combin'])
# #     pre_test = pre_test[['ID','item_cnt_day']].copy().rename(columns = {'item_cnt_day': 'item_cnt_month'})
#     return data_Next

# def get_error(pre):
#     pre = np.array(pre)
#     true = np.array(data_train_MoveAverage[data_train_MoveAverage["date_block_num"]==33].sort_values(by=['combin'])["item_cnt_day"].astype('float32'))
#     error = np.sum(np.square(pre - true))
#     return error
# pre_test.to_csv('submission.csv',index=None)

In [ ]:
# alpha_list = [round(0.1*x,1) for x in range(1,10,1)]
# error_lis = []
# for alpha in alpha_list:
#     data_pre = predict_test(alpha)
#     error = get_error(data_pre)
#     print(f"alpha:{alpha}",f"error:{error}")
#     error_lis.append([alpha,error])

In [ ]:
# alpha_list

### 参考下别人的代码

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

from itertools import product
from sklearn.preprocessing import LabelEncoder

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from xgboost import XGBRegressor
from xgboost import plot_importance

def plot_features(booster, figsize):    
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return plot_importance(booster=booster, ax=ax)

import time
import sys
import gc
import pickle
sys.version_info

In [ ]:
train_data.loc[train_data.item_price==10, 'item_price']
# 这个方式可以输出item_price的序列：loc是以行列名的方式索引，而iloc是以index的方式索引，前行后列

In [ ]:
# Якутск Орджоникидзе, 56
train_data.loc[train_data.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57
# Якутск ТЦ "Центральный"
train_data.loc[train_data.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58
# Жуковский ул. Чкалова 39м²
train_data.loc[train_data.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11

In [ ]:
# Each shop_name starts with the city name.
# Each category contains type and subtype in its name.
shops.head(5)

In [ ]:
shops['city'] = shops.shop_name.map(lambda x:str(x).split()[0])
shops.loc[shops.city == '!Якутск','city'] = 'Якутск'
# 对city进行label——encoding
shops['city_code'] = LabelEncoder().fit_transform(shops['city'] )
# 然后只要city_code 与 shop_id
shops = shops[['shop_id', 'city_code']]

In [ ]:
item_categories.head(5)

In [ ]:
def get_type(x):
    x1 = str(x["item_category_name"]).split('-')[0].strip()
    try:
        x2 = str(x["item_category_name"]).split('-')[1].strip()
    except:
        x2 = x1
    return pd.Series([x1,x2])
item_categories[["type","subtype"]] = item_categories.apply(get_type,axis=1)
# 编码
item_categories["type_code"] = LabelEncoder().fit_transform(item_categories["type"])
item_categories["subtype_code"] = LabelEncoder().fit_transform(item_categories["subtype"])
item_categories.drop(["item_category_name","type","subtype"],axis = 1, inplace=True)

In [ ]:
len(set(test.shop_id)),len(set(test.item_id)), len(set(test.item_id).intersection(set(train_data.item_id)))# 集合——交集运算

In [ ]:
#product(A,B)函数,返回A和B中的元素组成的笛卡尔积的元组
matrix = []
cols = ['date_block_num','shop_id','item_id']
for i in range(34):
    # 得到当月的销售数据
    sale = train_data[train_data.date_block_num == i]
    matrix.append(
    np.array(list(product([i],sale.shop_id.unique(), sale.item_id.unique())),dtype='int16')
    )
# 得到的matrix为list(nparray([(),(),...]))



In [ ]:
matrix = pd.DataFrame(np.vstack(matrix), columns=cols) #(10913804, 3)
#np.vstack() 按垂直方向（行顺序）堆叠数组构成一个新的数组


In [ ]:
matrix['date_block_num'] = matrix['date_block_num'].astype(np.int8)
matrix['shop_id'] = matrix['shop_id'].astype(np.int8)
matrix['item_id'] = matrix['item_id'].astype(np.int16)
matrix.sort_values(cols,inplace=True)

In [ ]:
train_data.columns,matrix.columns


In [ ]:
# 因为测试集中有的商品没有在训练集中，所以这些商品的销量预测可能为0，其次，需要知道训练集中的商品的信息
# 先聚合 再merge
# 已经聚合了，删除一些列
train_data.drop(['combin'],axis = 1,inplace = True)

In [ ]:
matrix = pd.merge(matrix, train_data, on=cols, how='left')
matrix['item_cnt_day'] = (matrix['item_cnt_day']
                                .fillna(0)
                                .clip(0,20) # 将值的范围限制在了0，20
                                .astype(np.float16))

In [ ]:
matrix.rename(columns = {'item_cnt_day':'item_cnt_month'},inplace = True)

In [ ]:
matrix.head(5)

In [ ]:
# 将测试集的数据加入到matrix中
test["date_block_num"] = 34
test["shop_id"] = test["shop_id"].astype(np.int8)
test["item_id"] = test["item_id"].astype(np.int16)
test["date_block_num"] = test["date_block_num"].astype(np.int8)

In [ ]:
matrix_new = pd.concat([matrix, test],axis=0,ignore_index=True, sort=False, keys=cols)

In [ ]:
# 对Shops/Items/Cats features进行merge
matrix_new = pd.merge(matrix_new,shops,how='left',on=['shop_id'])
matrix_new = pd.merge(matrix_new,items,how='left',on=['item_id'])
matrix_new = pd.merge(matrix_new,item_categories,how='left',on=['item_category_id'])
matrix_new['city_code'] = matrix_new['city_code'].astype(np.int8)
matrix_new['item_category_id'] = matrix_new['item_category_id'].astype(np.int8)
matrix_new['type_code'] = matrix_new['type_code'].astype(np.int8)
matrix_new['subtype_code'] = matrix_new['subtype_code'].astype(np.int8)

In [ ]:
matrix_new.drop(['year','month','ID','combin','item_name'],axis = 1,inplace=True)

In [ ]:
def lag_feature(df, lags, col):
    tmp = df[['date_block_num','shop_id','item_id',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id', col+'_lag_'+str(i)]
        shifted['date_block_num'] += i
        df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df

In [ ]:
matrix_new = lag_feature(matrix_new, [1,2,3,6,12], 'item_cnt_month')

In [ ]:
matrix_new

In [ ]:
#Mean encoded features
group = matrix_new.groupby(['date_block_num']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_avg_item_cnt' ]
group.reset_index(inplace=True) # 把index重新设置下



In [ ]:
group

In [ ]:
matrix_new = pd.merge(matrix_new, group, on=['date_block_num'], how='left')
matrix_new['date_avg_item_cnt'] = matrix_new['date_avg_item_cnt'].astype(np.float16)
matrix_new = lag_feature(matrix_new, [1], 'date_avg_item_cnt')
matrix_new.drop(['date_avg_item_cnt'], axis=1, inplace=True)
matrix_new

In [ ]:
# 对item_id进行编码
group = matrix_new.groupby(['date_block_num','item_id']).agg({'item_cnt_month':'mean'})
group.columns = ['date_item_avg_item_cnt']
group.reset_index(inplace = True) #重新设置index

In [ ]:
matrix_new = pd.merge(matrix_new,group,on=['date_block_num','item_id'],how = 'left')
matrix_new['date_item_avg_item_cnt'] = matrix_new['date_item_avg_item_cnt'].astype(np.float16)
matrix_new = lag_feature(matrix_new, [1,2,3,6,12], 'date_item_avg_item_cnt')
matrix_new.drop(['date_item_avg_item_cnt'], axis=1, inplace=True)

In [ ]:
matrix=matrix_new
group = matrix.groupby(['date_block_num', 'shop_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_shop_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','shop_id'], how='left')
matrix['date_shop_avg_item_cnt'] = matrix['date_shop_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1,2,3,6,12], 'date_shop_avg_item_cnt')
matrix.drop(['date_shop_avg_item_cnt'], axis=1, inplace=True)

In [ ]:
group = matrix.groupby(['date_block_num', 'item_category_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_cat_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','item_category_id'], how='left')
matrix['date_cat_avg_item_cnt'] = matrix['date_cat_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_cat_avg_item_cnt')
matrix.drop(['date_cat_avg_item_cnt'], axis=1, inplace=True)

In [ ]:
group = matrix.groupby(['date_block_num', 'shop_id', 'item_category_id']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_shop_cat_avg_item_cnt']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'shop_id', 'item_category_id'], how='left')
matrix['date_shop_cat_avg_item_cnt'] = matrix['date_shop_cat_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_shop_cat_avg_item_cnt')
matrix.drop(['date_shop_cat_avg_item_cnt'], axis=1, inplace=True)

In [ ]:
group = matrix.groupby(['date_block_num', 'shop_id', 'type_code']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_shop_type_avg_item_cnt']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'shop_id', 'type_code'], how='left')
matrix['date_shop_type_avg_item_cnt'] = matrix['date_shop_type_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_shop_type_avg_item_cnt')
matrix.drop(['date_shop_type_avg_item_cnt'], axis=1, inplace=True)

In [ ]:
group = matrix.groupby(['date_block_num', 'shop_id', 'subtype_code']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_shop_subtype_avg_item_cnt']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'shop_id', 'subtype_code'], how='left')
matrix['date_shop_subtype_avg_item_cnt'] = matrix['date_shop_subtype_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_shop_subtype_avg_item_cnt')
matrix.drop(['date_shop_subtype_avg_item_cnt'], axis=1, inplace=True)

In [ ]:
group = matrix.groupby(['date_block_num', 'city_code']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_city_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'city_code'], how='left')
matrix['date_city_avg_item_cnt'] = matrix['date_city_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_city_avg_item_cnt')
matrix.drop(['date_city_avg_item_cnt'], axis=1, inplace=True)

In [ ]:
group = matrix.groupby(['date_block_num', 'item_id', 'city_code']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_item_city_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'item_id', 'city_code'], how='left')
matrix['date_item_city_avg_item_cnt'] = matrix['date_item_city_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_item_city_avg_item_cnt')
matrix.drop(['date_item_city_avg_item_cnt'], axis=1, inplace=True)

In [ ]:
group = matrix.groupby(['date_block_num', 'type_code']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_type_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'type_code'], how='left')
matrix['date_type_avg_item_cnt'] = matrix['date_type_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_type_avg_item_cnt')
matrix.drop(['date_type_avg_item_cnt'], axis=1, inplace=True)

In [ ]:
group = matrix.groupby(['date_block_num', 'subtype_code']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_subtype_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'subtype_code'], how='left')
matrix['date_subtype_avg_item_cnt'] = matrix['date_subtype_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_subtype_avg_item_cnt')
matrix.drop(['date_subtype_avg_item_cnt'], axis=1, inplace=True)

In [ ]:
train_data.columns

In [ ]:
# ['item_avg_item_price', 'date_item_avg_item_price']
# item_avg_item_price：itemid price mean
# date_item_avg_item_price: itemid date price mean
group = train_data.groupby(['item_id']).agg({'item_price':'mean'})
group.columns = ['item_avg_item_price']
group.reset_index(inplace = True)

matrix = pd.merge(matrix, group, on = ['item_id'],how = 'left')
matrix['item_avg_item_price'] = matrix['item_avg_item_price'].astype(np.float16)


group = train_data.groupby(['date_block_num','item_id']).agg({'item_price':'mean'})
group.columns = ['date_item_avg_item_price']
group.reset_index(inplace = True)

matrix = pd.merge(matrix, group, on = ['date_block_num','item_id'],how='left')
matrix['date_item_avg_item_price'] = matrix['date_item_avg_item_price'].astype(np.float16)

lags = [1,2,3,4,5,6]
matrix = lag_feature(matrix, lags, 'date_item_avg_item_price')

for i in lags:
    matrix['delta_price_lag_'+str(i)] = \
        (matrix['date_item_avg_item_price_lag_'+str(i)] - matrix['item_avg_item_price']) / matrix['item_avg_item_price']

def select_trend(row):
    for i in lags:
        if row['delta_price_lag_'+str(i)]:
            return row['delta_price_lag_'+str(i)]
    return 0
    
matrix['delta_price_lag'] = matrix.apply(select_trend, axis=1)
matrix['delta_price_lag'] = matrix['delta_price_lag'].astype(np.float16)
matrix['delta_price_lag'].fillna(0, inplace=True)

fetures_to_drop = ['item_avg_item_price', 'date_item_avg_item_price']
for i in lags:
    fetures_to_drop += ['date_item_avg_item_price_lag_'+str(i)]
    fetures_to_drop += ['delta_price_lag_'+str(i)]

matrix.drop(fetures_to_drop, axis=1, inplace=True)

In [ ]:
group = train_data.groupby(['date_block_num','shop_id']).agg({'total_price': ['sum']})
group.columns = ['date_shop_revenue']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','shop_id'], how='left')
matrix['date_shop_revenue'] = matrix['date_shop_revenue'].astype(np.float32)

group = group.groupby(['shop_id']).agg({'date_shop_revenue': ['mean']})
group.columns = ['shop_avg_revenue']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['shop_id'], how='left')
matrix['shop_avg_revenue'] = matrix['shop_avg_revenue'].astype(np.float32)

matrix['delta_revenue'] = (matrix['date_shop_revenue'] - matrix['shop_avg_revenue']) / matrix['shop_avg_revenue']
matrix['delta_revenue'] = matrix['delta_revenue'].astype(np.float16)

matrix = lag_feature(matrix, [1], 'delta_revenue')

matrix.drop(['date_shop_revenue','shop_avg_revenue','delta_revenue'], axis=1, inplace=True)

In [ ]:
matrix['month'] = matrix['date_block_num'] % 12

In [ ]:
days = pd.Series([31,28,31,30,31,30,31,31,30,31,30,31])
matrix['days'] = matrix['month'].map(days).astype(np.int8)

In [ ]:
ts = time.time()
cache = {}
matrix['item_shop_last_sale'] = -1
matrix['item_shop_last_sale'] = matrix['item_shop_last_sale'].astype(np.int8)
for idx, row in matrix.iterrows():    
    key = str(row.item_id)+' '+str(row.shop_id)
    if key not in cache:
        if row.item_cnt_month!=0:
            cache[key] = row.date_block_num
    else:
        last_date_block_num = cache[key]
        matrix.at[idx, 'item_shop_last_sale'] = row.date_block_num - last_date_block_num
        cache[key] = row.date_block_num         
time.time() - ts

In [ ]:
ts = time.time()
cache = {}
matrix['item_last_sale'] = -1
matrix['item_last_sale'] = matrix['item_last_sale'].astype(np.int8)
for idx, row in matrix.iterrows():    
    key = row.item_id
    if key not in cache:
        if row.item_cnt_month!=0:
            cache[key] = row.date_block_num
    else:
        last_date_block_num = cache[key]
        if row.date_block_num>last_date_block_num:
            matrix.at[idx, 'item_last_sale'] = row.date_block_num - last_date_block_num
            cache[key] = row.date_block_num         
time.time() - ts

In [ ]:
ts = time.time()
matrix['item_shop_first_sale'] = matrix['date_block_num'] - matrix.groupby(['item_id','shop_id'])['date_block_num'].transform('min')
matrix['item_first_sale'] = matrix['date_block_num'] - matrix.groupby('item_id')['date_block_num'].transform('min')
time.time() - ts

In [ ]:
ts = time.time()
matrix = matrix[matrix.date_block_num > 11]
time.time() - ts

In [ ]:
ts = time.time()
def fill_na(df):
    for col in df.columns:
        if ('_lag_' in col) & (df[col].isnull().any()):
            if ('item_cnt' in col):
                df[col].fillna(0, inplace=True)         
    return df

matrix = fill_na(matrix)
time.time() - ts

In [ ]:
matrix.columns

In [ ]:
# matrix.drop(['item_price','total_price'],axis = 1,inplace = True)
matrix.drop(['item_avg_item_price_x','item_avg_item_price_y'],axis = 1,inplace = True)

In [ ]:
matrix.info()

In [ ]:
matrix.to_pickle('data.pkl')
del matrix
del matrix_new
del cache
del group
del items
del shops
del item_categories
del train_data
del train_data_test
# leave test for submission
gc.collect();

In [ ]:
data = pd.read_pickle('data.pkl')

In [ ]:
data = data[[
    'date_block_num',
    'shop_id',
    'item_id',
    'item_cnt_month',
    'city_code',
    'item_category_id',
    'type_code',
    'subtype_code',
    'item_cnt_month_lag_1',
    'item_cnt_month_lag_2',
    'item_cnt_month_lag_3',
    'item_cnt_month_lag_6',
    'item_cnt_month_lag_12',
    'date_avg_item_cnt_lag_1',
    'date_item_avg_item_cnt_lag_1',
    'date_item_avg_item_cnt_lag_2',
    'date_item_avg_item_cnt_lag_3',
    'date_item_avg_item_cnt_lag_6',
    'date_item_avg_item_cnt_lag_12',
    'date_shop_avg_item_cnt_lag_1',
    'date_shop_avg_item_cnt_lag_2',
    'date_shop_avg_item_cnt_lag_3',
    'date_shop_avg_item_cnt_lag_6',
    'date_shop_avg_item_cnt_lag_12',
    'date_cat_avg_item_cnt_lag_1',
    'date_shop_cat_avg_item_cnt_lag_1',
    #'date_shop_type_avg_item_cnt_lag_1',
    #'date_shop_subtype_avg_item_cnt_lag_1',
    'date_city_avg_item_cnt_lag_1',
    'date_item_city_avg_item_cnt_lag_1',
    #'date_type_avg_item_cnt_lag_1',
    #'date_subtype_avg_item_cnt_lag_1',
    'delta_price_lag',
    'month',
    'days',
    'item_shop_last_sale',
    'item_last_sale',
    'item_shop_first_sale',
    'item_first_sale',
]]

In [ ]:
X_train = data[data.date_block_num < 33].drop(['item_cnt_month'], axis=1)
Y_train = data[data.date_block_num < 33]['item_cnt_month']
X_valid = data[data.date_block_num == 33].drop(['item_cnt_month'], axis=1)
Y_valid = data[data.date_block_num == 33]['item_cnt_month']
X_test = data[data.date_block_num == 34].drop(['item_cnt_month'], axis=1)

In [ ]:
del data
gc.collect();

In [ ]:
ts = time.time()

model = XGBRegressor(
    max_depth=8,
    n_estimators=1000,
    min_child_weight=300, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=0.3,    
    seed=42)

model.fit(
    X_train, 
    Y_train, 
    eval_metric="rmse", 
    eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
    verbose=True, 
    early_stopping_rounds = 10)

time.time() - ts

In [ ]:
Y_pred = model.predict(X_valid).clip(0, 20)
Y_test = model.predict(X_test).clip(0, 20)

submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": Y_test
})
submission.to_csv('submission.csv', index=False)

# save predictions for an ensemble
pickle.dump(Y_pred, open('xgb_train.pickle', 'wb'))
pickle.dump(Y_test, open('xgb_test.pickle', 'wb'))

In [ ]:
plot_features(model, (10,14))